In [2]:
import sys
sys.path.append('../')
sys.path.append('/Users/adam2392/Documents/fragility_analysis/')
import fragility.util.utils as futil
import datainterface.patient as Pat
# from fragility.signalprocessing import frequencyanalysis

import os
import time

import numpy as np
import pandas as pd
import scipy
import scipy.io

import processing.util as util
import processing.frequencytransform as ft
import peakdetect
import processing.preprocessfft as preprocess

# sys.path.append('/Users/adam2392/Documents/tvb/')
# sys.path.append('/Users/adam2392/Documents/tvb/_tvbdata/')
# sys.path.append('/Users/adam2392/Documents/tvb/_tvblibrary/')
# from tvb.simulator.lab import *
# import tvbsim.util

from natsort import natsorted
import ntpath
from sklearn.decomposition import PCA

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

def _gettimepoints(numsignals, numwinsamps, numstepsamps):
    # create array of indices of window start times
    timestarts = np.arange(0, numsignals-numwinsamps+1, numstepsamps)
    # create array of indices of window end times
    timeends = np.arange(numwinsamps-1, numsignals, numstepsamps)
    # create the timepoints array for entire data array
    timepoints = np.append(timestarts.reshape(len(timestarts), 1), timeends.reshape(len(timestarts), 1), axis=1)
    return timepoints

In [4]:
expname = 'cnn'

# dirs for the training data from the model 
datadir = os.path.join('/Volumes/ADAM LI/pydata/dnn/', expname)
tempdir = os.path.join(datadir, '_temp')
finaldir = os.path.join(datadir, '_final')

# the original raw data dir
metadatadir = '/Volumes/ADAM LI/pydata/metadata/'    
traindir = os.path.join('/Volumes/ADAM LI/pydata/traindata/fft/', expname)

# Get ALL datafiles from all downstream files
datafiles = []
for root, dirs, files in os.walk(datadir):
    for file in files:
        datafiles.append(os.path.join(root, file))
print(len(datafiles))
# print(datafiles[50:])
# print(datafiles)

0


# Visualize Final History

Investigate the history object saved at the final point in time and see how training proceeded. Get a value of the accuracy and precision of the neural network.

Visualize any other important metadata.

# Visualize The Diversity of the Data That We Used

Visualize the diversity of the data we used to train the model.